In [2]:
#code taken from https://github.com/madelinehayes/seabirdNET/blob/master/uas_img_handler_FINAL.ipynb

In [1]:
from PIL import Image
import os
from os import walk
import argparse
import numpy as np
import json
import csv
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.windows import Window
import matplotlib
import folium
from pyproj import Proj, transform

%matplotlib inline

Image.MAX_IMAGE_PIXELS = 100000000000

In [4]:
#access all elephant seal imagery
seal = 'Elephant_Seals'
sealPath = '../Images/' + seal 

overlap = 80
only_beach = False    
    
files = next(walk(sealPath), (None, None, []))[2]  # empty if no files exist

In [3]:
def checkImageSize(seal, image):
    infile = '../Images/' + seal +'/' + image
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    return img_height, img_width
    
    
sizes = set()
for image in files:
    size = checkImageSize(seal, image)
    sizes.add(size)
    
print(len(sizes))
assert(len(sizes) == 1)

C:\Users\mgiff\anaconda3\envs\rasterio\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


1


In [7]:
if True:
    print(sizes)
    for size in sizes:
        img_height, img_width = size
        tile_height = tile_width = 500
        stride = tile_height - overlap
        for r in range(0, img_height-tile_height+1, stride):
            for c in range(0, img_width-tile_width+1, stride):
                print(r, c) #row, column
            print()

{(3352, 4800)}
0 0
0 420
0 840
0 1260
0 1680
0 2100
0 2520
0 2940
0 3360
0 3780
0 4200

420 0
420 420
420 840
420 1260
420 1680
420 2100
420 2520
420 2940
420 3360
420 3780
420 4200

840 0
840 420
840 840
840 1260
840 1680
840 2100
840 2520
840 2940
840 3360
840 3780
840 4200

1260 0
1260 420
1260 840
1260 1260
1260 1680
1260 2100
1260 2520
1260 2940
1260 3360
1260 3780
1260 4200

1680 0
1680 420
1680 840
1680 1260
1680 1680
1680 2100
1680 2520
1680 2940
1680 3360
1680 3780
1680 4200

2100 0
2100 420
2100 840
2100 1260
2100 1680
2100 2100
2100 2520
2100 2940
2100 3360
2100 3780
2100 4200

2520 0
2520 420
2520 840
2520 1260
2520 1680
2520 2100
2520 2520
2520 2940
2520 3360
2520 3780
2520 4200



In [5]:
def processImage(seal, image, overlap):
    '''
    args:
    seal is a string, either "Harbor_Seals" or "Elephant_Seals"
    image is the satellite image file
    overlap is an int with the size of overlap between images
    '''
    infile = '../Images/' + seal +'/' + image
    #img_dir = '..' + infile.split(".")[2] + '_500x500_overlap' + str(overlap)
    img_dir = '../Images/' + seal 
    if only_beach:
        img_dir += '_Beach_Only'
    else:
        img_dir += '_Full_Data'
    prj_name = img_dir.split("/")[-1]
    
    #open file (read), get dims
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    
    # define tile size and number of pixels to move in each direction
    tile_height = tile_width = 500
    stride = tile_height - overlap
    start_num = 0
    img_dict = {}
    count = 0
    
    # create the dir if it doesn't already exist
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
        
    num_tiles = (img_width * img_height) / ((tile_height - overlap) * (tile_width - overlap))
    #print("image size", dataset.shape)
    #print("number of tiles:", num_tiles)
    
    for r in range(0, img_height-tile_height+1, stride):
        for c in range(0, img_width-tile_width+1, stride):
            # read tile
            tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
            
            reshaped_tile = reshape_as_image(tile)
            
            top_pixel = [c,r]
            imag = image.split(".")[0]
            tile_name = imag + "---" + str(count) + ".png"
            img_dict[tile_name] = top_pixel
            
            im = Image.fromarray(reshaped_tile)
            #save 
            path = os.path.join(img_dir, tile_name)
            im.save(path)
            
            count += 1
            
    '''
    not sure if the following is necessary...
    
    full_dict = {"image_name" : infile,
            "image_locations" : img_dict,
             "crs" : str(dataset.crs)
            }
    with open(img_dir + '/lastLine.json', 'w') as fp:
        json.dump(full_dict, fp)
    '''

In [8]:
countI = 1
for image in files:
    processImage(seal, image, overlap)
    print('finished image', countI)
    countI += 1

finished image 1
finished image 2
finished image 3
finished image 4
finished image 5
finished image 6
finished image 7
finished image 8
finished image 9
finished image 10
finished image 11
finished image 12
finished image 13
finished image 14
finished image 15
finished image 16
finished image 17
finished image 18
finished image 19
finished image 20
finished image 21
finished image 22
finished image 23
finished image 24
finished image 25
finished image 26
finished image 27
finished image 28
finished image 29
finished image 30
finished image 31
finished image 32
finished image 33
finished image 34
finished image 35
finished image 36
finished image 37
finished image 38
finished image 39
finished image 40
finished image 41
finished image 42
finished image 43
finished image 44
finished image 45
finished image 46
finished image 47
finished image 48
finished image 49
finished image 50
finished image 51
finished image 52
finished image 53
finished image 54
finished image 55
finished image 56
f

In [9]:
print("done")

done


In [10]:
# get images on right and bottom
def get_right_bottom_image(seal, image):
    infile = '../Images/' + seal +'/' + image
    img_dir = '../Images/' + seal 
    if only_beach:
        img_dir += '_Beach_Only'
    else:
        img_dir += '_Full_Data'
    prj_name = img_dir.split("/")[-1]
    
    #open file (read), get dims
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    
    # define tile size and number of pixels to move in each direction
    tile_height = tile_width = 500
    #stride = tile_height - overlap
    start_num = 0
    img_dict = {}
    count = 0
    
    far_x = img_width - 500
    bottom_y = img_height - 500
    
    # create the dir if it doesn't already exist
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
        
    num_tiles = (img_width * img_height) / ((tile_height - overlap) * (tile_width - overlap))
    #print("image size", dataset.shape)
    #print("number of tiles:", num_tiles)
    count = int(num_tiles)
    r = 0
    c = far_x
    stride = tile_height - overlap
    #get the right most column... will have some extra overlap with previously tiled tiles
    while r + tile_height < img_height:
        #print(r, c)
        tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
        
        reshaped_tile = reshape_as_image(tile)
            
        top_pixel = [c,r]
        imag = image.split(".")[0]
        tile_name = imag + "---" + str(count) + ".png"
        img_dict[tile_name] = top_pixel

        im = Image.fromarray(reshaped_tile)
        #save 
        path = os.path.join(img_dir, tile_name)
        im.save(path)

        count += 1    
        r += stride
    
    r = bottom_y
    c = 0
    while c + tile_width < img_width:
        #print(r,c)
        tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
        
        reshaped_tile = reshape_as_image(tile)
            
        top_pixel = [c,r]
        imag = image.split(".")[0]
        tile_name = imag + "---" + str(count) + ".png"
        img_dict[tile_name] = top_pixel

        im = Image.fromarray(reshaped_tile)
        #save 
        path = os.path.join(img_dir, tile_name)
        im.save(path)

        count += 1    
        c += stride

In [11]:
countI = 1
for image in files:
    get_right_bottom_image(seal, image)
    print('finished image', countI)
    countI += 1

finished image 1
finished image 2
finished image 3
finished image 4
finished image 5
finished image 6
finished image 7
finished image 8
finished image 9
finished image 10
finished image 11
finished image 12
finished image 13
finished image 14
finished image 15
finished image 16
finished image 17
finished image 18
finished image 19
finished image 20
finished image 21
finished image 22
finished image 23
finished image 24
finished image 25
finished image 26
finished image 27
finished image 28
finished image 29
finished image 30
finished image 31
finished image 32
finished image 33
finished image 34
finished image 35
finished image 36
finished image 37
finished image 38
finished image 39
finished image 40
finished image 41
finished image 42
finished image 43
finished image 44
finished image 45
finished image 46
finished image 47
finished image 48
finished image 49
finished image 50
finished image 51
finished image 52
finished image 53
finished image 54
finished image 55
finished image 56
f

In [12]:
print("done")

done
